In [1]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()
SILICONFLOW_API_KEY = os.getenv("SILICONFLOW_API_KEY")

In [2]:
import re

def clean_text(text):
	"""
	清理文本内容，去除Wiki标记和多余格式。
	"""
	text = re.sub(r'\{\{对话\|([^|]+)\|([^}]+)\}\}', lambda m: f"{m.group(1)}：{m.group(2).replace('<br>', ' ')}", text)

	text = re.sub(r'\{\{(?:xh|xl|状态|物品|任务|货币|版本|地图)\|([^|}]*)[^}]*\}\}', r'\1', text)

	text = re.sub(r'\{\{Role\|[^|]+\|([^}]+)\}\}', r'\1', text)

	text = re.sub(r'\{\{[^}]+text=([^|}]+)[^}]*\}\}', r'\1', text)

	text = re.sub(r'\{\{黑幕\|([^}]+)\}\}', r'\1', text)
	text = re.sub(r'<span[^>]*class="blackcover"[^>]*>(.*?)</span>', r'\1', text)

	text = re.sub(r'\[\[(?:[^|\]]+\:)?([^|\]]+\|)?([^\]]+)\]\]', lambda m: m.group(2) if m.group(2) else m.group(1).split('|')[0], text)

	text = re.sub(r'<[^>]+>', '', text)
	text = re.sub(r"'''", '', text) 
	text = re.sub(r"''", '', text)

	text = re.sub(r'\{\{[^}]+\}\}', '', text)

	text = re.sub(r'={2,}([^=]+)={2,}', r'\1', text)
	text = re.sub(r'^\*+', '· ', text, flags=re.MULTILINE)

	lines = [line.strip() for line in text.splitlines() if line.strip()]
	return "\n".join(lines)

def process_docs(input_dir, output_dir):
	"""
	处理输入目录中的所有文本文件，清理内容后保存到输出目录。
	"""
	if not os.path.exists(output_dir):
		os.makedirs(output_dir)

	for filename in os.listdir(input_dir):
		if filename.endswith(".txt"):
			with open(os.path.join(input_dir, filename), 'r', encoding='utf-8') as f:
				content = f.read()
			
			clean_content = clean_text(content)
			
			with open(os.path.join(output_dir, filename), 'w', encoding='utf-8') as f:
				f.write(clean_content)
			print(f"成功清洗: {filename}")

In [3]:
RAW_DIR = "./docs_raw"
CLEAN_DIR = "./docs_clean"

if os.path.exists(RAW_DIR):
	process_docs(RAW_DIR, CLEAN_DIR)
	print("--- 清洗完成 ---")
else:
	print(f"错误: 找不到源文件夹 {RAW_DIR}")

成功清洗: doc1.txt
成功清洗: doc2.txt
--- 清洗完成 ---


In [4]:
# 准备向量化与 RAG 调用
import requests
import json
import numpy as np


In [5]:
# BGE-M3 向量化函数
def get_embedding(text):
	"""调用 BGE-M3 获取文本向量"""
	url = "https://api.siliconflow.cn/v1/embeddings"
	headers = {
		"Authorization": f"Bearer {SILICONFLOW_API_KEY}",
		"Content-Type": "application/json"
	}
	payload = {
		"model": "BAAI/bge-m3",
		"input": [text]
	}
	resp = requests.post(url, headers=headers, json=payload)
	data = resp.json()
	return data["data"][0]["embedding"]

# 测试
sample_text = "Agentic RAG 是结合检索与生成的系统"
vector = get_embedding(sample_text)
print(f"向量长度: {len(vector)}")


向量长度: 1024


In [6]:
def chunk_text(text, min_len=100, max_len=1500):
	"""
	按换行符切分文本，每块长度控制在 min_len ~ max_len 之间。
	
	- min_len: 避免太短的段落
	- max_len: 避免超过模型 token 限制
	"""
	lines = [line.strip() for line in text.split("\n") if line.strip()]
	chunks = []
	current_chunk = ""
	
	for line in lines:
		if len(current_chunk) + len(line) + 1 <= max_len:
			current_chunk += line + "\n"
		else:
			if len(current_chunk) >= min_len:
				chunks.append(current_chunk.strip())
			current_chunk = line + "\n"
	
	if current_chunk.strip():
		chunks.append(current_chunk.strip())
	
	return chunks


In [7]:
from chromadb import Client, PersistentClient
from chromadb.config import Settings
import os

DB_DIR = "./chroma_db"
COLLECTION_NAME = "text_chunks"

client = PersistentClient(path=DB_DIR)

# 判断是否已有可用向量库
db_exists = os.path.exists(DB_DIR)
collection_names = [c.name for c in client.list_collections()]

if db_exists and COLLECTION_NAME in collection_names:
    collection = client.get_collection(name=COLLECTION_NAME)
    if collection.count() > 0:
        print(f"检测到本地 ChromaDB，已包含 {collection.count()} 个向量，跳过构建")
    else:
        print("集合存在但为空，开始构建向量库")
else:
    print("未检测到本地向量库，开始构建")

# 只有在需要时才构建
if not (db_exists and COLLECTION_NAME in collection_names and collection.count() > 0):
    if COLLECTION_NAME in collection_names:
        collection = client.get_collection(name=COLLECTION_NAME)
    else:
        collection = client.create_collection(name=COLLECTION_NAME)

    for fname in os.listdir(CLEAN_DIR):
        if not fname.endswith(".txt"):
            continue

        with open(os.path.join(CLEAN_DIR, fname), "r", encoding="utf-8") as f:
            text = f.read()

        for i, chunk in enumerate(chunk_text(text)):
            vec = get_embedding(chunk)
            if vec is not None:
                collection.add(
                    ids=[f"{fname}_part{i+1}"],
                    documents=[chunk],
                    embeddings=[vec]
                )

    
    print(f"ChromaDB 向量库构建完成，共 {collection.count()} 个文档块")


检测到本地 ChromaDB，已包含 27 个向量，跳过构建


In [8]:
import requests
import numpy as np

# 检索 + BGE-Reranker 重排
def retrieve(query, top_k=5):
	query_vec = get_embedding(query)
	
	# ChromaDB 初步检索 top_k
	results = collection.query(
		query_embeddings=[query_vec],
		n_results=top_k
	)
	
	# 构建 top_docs
	top_docs = []
	if results['documents']:
		for doc_id, text in zip(results['ids'][0], results['documents'][0]):
			top_docs.append({
				"id": doc_id,
				"text": text
			})
	
	# 调用 BGE-Reranker 进一步重排
	url = "https://api.siliconflow.cn/v1/rerank"
	headers = {
		"Authorization": f"Bearer {SILICONFLOW_API_KEY}",
		"Content-Type": "application/json"
	}
	payload = {
		"model": "BAAI/bge-reranker-v2-m3",
		"query": query,
		"documents": [doc["text"] for doc in top_docs]
	}
	resp = requests.post(url, headers=headers, json=payload)
	data = resp.json()
	
	if "results" in data and data["results"]:
		# 按相关性降序排序
		sorted_indices = [
			item["index"] for item in sorted(
				data["results"],
				key=lambda x: x["relevance_score"],
				reverse=True
			)
		]
		reranked_docs = [top_docs[i] for i in sorted_indices]
	else:
		reranked_docs = top_docs

	return reranked_docs

# 测试
top_docs = retrieve("什么是 Agentic RAG？")
for i, doc in enumerate(top_docs):
	print(f"Top-{i+1} 文档: {doc['id']}")


Top-1 文档: doc1.txt_part13
Top-2 文档: doc1.txt_part18
Top-3 文档: doc1.txt_part15
Top-4 文档: doc1.txt_part16
Top-5 文档: doc1.txt_part14


In [9]:
# 调用 DeepSeek-R1 生成答案
def generate_answer(query, context_docs):
	url = "https://api.siliconflow.cn/v1/chat/completions"
	headers = {
		"Authorization": f"Bearer {SILICONFLOW_API_KEY}",
		"Content-Type": "application/json"
	}
	
	context_text = "\n".join([doc["text"] for doc in context_docs])
	prompt = f"""
请根据以下参考内容回答问题，只输出最终答案，不要输出任何思考过程或分析步骤。
严格按照条列格式输出，禁止任何解释或推理。

【参考内容】
{context_text}

【问题】
{query}

【要求】
- 输出必须为 Markdown 条列形式，并且尽可能简洁
- 不允许出现任何“我认为 / 首先 / 我需要思考 / 接下来 / 为了确认”等语言
"""
	
	payload = {
		"model": "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
		"messages": [
			{
				"role": "system",
				"content": "你只输出最终答案，不输出任何思考过程。"
			},
			{
				"role": "user",
				"content": prompt
			}
		],
		"max_tokens": 2000,
		"temperature": 0.2
	}

	resp = requests.post(url, headers=headers, json=payload)
	content = resp.json()["choices"][0]["message"]["content"]
	
	if "</think>" in content:
		answer = content.split("</think>")[-1].strip()
	else:
		answer = content.strip()
	
	return answer

# 测试
answer = generate_answer("什么是 Agentic RAG？", top_docs)
print("生成答案:\n", answer)


生成答案:
 **Agentic RAG**

- **Agentic**: 主动的、控制的  
- **RAG**: 可能指某种角色或技能组合  
- 结合上下文，可能指一种战斗机制或技能组合，用于主动控制或触发特定效果


In [10]:
# 一步调用 RAG
def rag_pipeline(query):
	# 1. 检索
	top_docs = retrieve(query)
	# 2. 生成答案
	answer = generate_answer(query, top_docs)
	return answer

# 测试
query = "Agentic RAG 的核心流程是什么？"
answer = rag_pipeline(query)
print("RAG 答案:\n", answer)


RAG 答案:
 Agentic RAG的核心流程可能包括以下步骤：

1. **初始化**：玩家或系统进入RAG环境，准备开始任务。
2. **资源收集**：玩家通过探索、战斗或任务完成获得资源。
3. **任务分配**：根据资源情况，系统或玩家决定执行的任务。
4. **任务执行**：玩家或系统执行分配的任务，可能涉及战斗、资源利用等。
5. **反馈与调整**：任务完成后，获得反馈并调整后续流程。

以上流程基于常见机制的推测，具体细节可能因实际游戏或系统而异。


In [11]:
query = "‘G17 6张 底30w 挂件v10’是什么意思？"
answer = rag_pipeline(query)
print("RAG 答案:\n", answer)

RAG 答案:
 G17 6张 底30w 挂件v10  
- G17：特定地图或副本  
- 6张：6个相关道具或副本  
- 底30w：底价30万金币  
- 挂件v10：第10版本的挂件道具


In [12]:
query = "月读极神的核心机制是什么？"
answer = rag_pipeline(query)
print("RAG 答案:\n", answer)

RAG 答案:
 ### 月读极神的核心机制

1. **极月读机制**：
   - 在P3阶段，赋予玩家四层新月或满月。
   - 场地被分为黑（新月）和白（满月）两色区域。
   - 黑区域会积累怨念，导致死亡宣告。
   - 白区域会附带15秒的出血和1秒的死亡宣告。

2. **百月光机制**：
   - 生成两个并排的月光释放范围可见的环形AOE。
   - 百月光会在两个位置释放，且位置必定在月牙的中心。

3. **深宵换装机制**：
   - 第一次极月读的深宵换装必定为火枪。
   - 第二次极月读的深宵换装必定为长枪。
   - 月相属性相反，即第一次为新月，第二次为满月，或反之。
